In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

/Users/tanaykarmarkar/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [34]:
from PROCESSING import create_labels

In [35]:
fetures, classes = create_labels('data.json')

In [39]:
test_size = 0.1
total_test = int(test_size*len(fetures))
train_x = fetures[:-total_test]
test_x = fetures[total_test:]
train_y = classes[:-total_test]
test_y = classes[total_test:]

In [14]:
nodes_hidden_layer_1 = 500
nodes_hidden_layer_2 = 600
nodes_hidden_layer_3 = 700
nodes_hidden_layer_4 = 100

In [48]:
num_classes = len(classes[0])
batch_size = 100

In [45]:
X = tf.placeholder('float',[None,len(train_x[0])])
Y = tf.placeholder('float')

In [46]:
def neural_net(Dataset_input):
    
    hidden_layer_1 = {'weights':tf.Variable(tf.random_normal([len(train_x[0]), nodes_hidden_layer_1]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_1]))}
    
    hidden_layer_2 = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_1, nodes_hidden_layer_2]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_2]))}
    
    hidden_layer_3 = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_2, nodes_hidden_layer_3]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_3]))}
    
    hidden_layer_4 = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_3, nodes_hidden_layer_4]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_4]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_4, num_classes]))
                     , 'bias':tf.Variable(tf.random_normal([num_classes]))}
    
    '''Defining Computation'''
    
    layer_1 = tf.add(tf.matmul(X,hidden_layer_1['weights']), hidden_layer_1['bias'])
    #Applying activation function
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1,hidden_layer_2['weights']), hidden_layer_2['bias'])
    layer_2 = tf.nn.relu(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2,hidden_layer_3['weights']), hidden_layer_3['bias'])
    layer_3 = tf.nn.relu(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3,hidden_layer_4['weights']), hidden_layer_4['bias'])
    layer_4 = tf.nn.relu(layer_4)
    
    output = tf.add(tf.matmul(layer_4,output_layer['weights']), output_layer['bias'])
    print('neural ok!')
    return output

In [54]:
def train(X):
    
    pred = neural_net(X)
    
    cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred,labels=Y))
    '''Mininmizing cost function'''
    # Using Gradient Descent
    optimizer = tf.train.AdamOptimizer().minimize(cost_function)
    
    
    Num_Epochs = 6
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer()) # All it does is initialize the variables, weights and bias what we need to initialize
        
        for epoch in range(Num_Epochs):
            loss = 0
            
            i = 0
            while i<len(train_x):
                start = i
                end = i+len(train_x[0])
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                n, c = sess.run([optimizer, cost_function], feed_dict ={X:batch_x,Y:batch_y})
                loss +=c #adding cost 
                i+=batch_size
            print('Epoch ', epoch, 'out of ', Num_Epochs, 'loss: ', loss)
        summary = tf.summary.FileWriter('log/sentiment-analysis',sess.graph)
        # tf.argmax ---> returns index of the maximum value, below line is just verifying that we are getting no more than 1 as output because it is one hot encoded
        check = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
        # tf.cast just changes to the dtype
        accuracy = tf.reduce_mean(tf.cast(check, tf.float32))
        print('Accuracy: ', accuracy.eval({X:test_x, Y:test_y}))

In [2]:
data = pd.read_json('data.json')

In [4]:
data.reset_index(inplace=True)

In [6]:
del data['index']

In [14]:
data['INPUTS'].to_csv('trainfrom.txt', index=False,header=None)

In [16]:
test_size = int(0.2*len(data))
train_x = data.INPUTS.iloc[:-test_size]
train_y = data.TARGET.iloc[:-test_size]
test_x = data.INPUTS.iloc[-test_size:]
test_y = data.TARGET.iloc[-test_size:]

In [18]:
with open('train.from', 'a', encoding='utf-8') as f:
    for content in train_x.values:
        f.write(content+'\n')

In [19]:
with open('train.to', 'a', encoding='utf-8') as f:
    for content in train_y.values:
        f.write(content+'\n')

In [20]:
with open('test.from', 'a', encoding='utf-8') as f:
    for content in test_x.values:
        f.write(content+'\n')

In [21]:
with open('test.to', 'a', encoding='utf-8') as f:
    for content in test_y.values:
        f.write(content+'\n')